In [1]:
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
#load data 
world_cup = pd.read_csv('Previous-Years T20-World-CupDataset.csv')
results = pd.read_csv('Past-Matches-results.csv')

In [3]:
world_cup.head(12)

,Team,Previous appearances,Previous \ntitles,Previous\n finals,Previous\n semifinals,Current \n rank
0,India,7,1,2,3,1
1,South Africa,7,0,0,2,3
2,Bangladesh,7,0,0,0,8
3,Zimbabwe,5,0,0,0,10
4,Pakistan,7,1,2,5,4
5,Netherland,4,0,0,0,12
6,New Zealand,7,0,1,3,5
7,England,7,1,2,3,2
8,Australia,7,1,2,4,6
9,Sri Lanka,7,1,2,4,7


In [4]:
results.head()

,Date,Team1,Team2,Winner,Venue
0,2-18-2022,Sri Lanka,Australia,Australia,Melbourne Cricket Ground
1,2-18-2022,India,West Indies,India,Eden Gardens
2,2-18-2022,Nepal,Oman,Nepal,Al Amerat Cricket Ground Oman Cricket (Ministr...
3,2-18-2022,Canada,Philippines,Canada,Al Amerat Cricket Ground Oman Cricket (Ministr...
4,2-18-2022,UAE,Ireland,United Arab Emirates,Al Amerat Cricket Ground Oman Cricket (Ministr...


In [5]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England ',' England ',' England',
                  ' South Africa ', ' South Africa', 'South Africa ',
                  ' Netherlands',' Netherlands ','Netherlands ',
                  ' Zimbabwe', ' Zimbabwe ','Zimbabwe ', 
            ' Pakistan', ' New Zealand', ' Sri Lanka', ' Afghanistan',  
            ' Pakistan ', ' New Zealand ', ' Sri Lanka ', ' Afghanistan ', 
            'Pakistan ', 'New Zealand ', 'Sri Lanka ', 'Afghanistan ', 
            ' Australia', ' Bangladesh', ' India', ' Ireland',
            ' Australia ', ' Bangladesh ', ' India ', ' Ireland ',
            'Australia ', 'Bangladesh ', 'India ', 'Ireland ']
df_teams_1 = results[results['Team1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

Date      1571
Team1     1571
Team2     1571
Winner    1571
Venue     1571
dtype: int64

In [6]:
df_teams.head(1571)

,Date,Team1,Team2,Winner,Venue
0,2-18-2022,Sri Lanka,Australia,Australia,Melbourne Cricket Ground
1,2-18-2022,India,West Indies,India,Eden Gardens
7,2-15-2022,Sri Lanka,Australia,Australia,Manuka Oval
9,2-14-2022,Ireland,Nepal,Ireland,Al Amerat Cricket Ground Oman Cricket (Ministr...
10,2-13-2022,Australia,Sri Lanka,tie,Sydney Cricket Ground
...,...,...,...,...,...
1286,2-24-2006,South Africa,Australia,South Africa,New Wanderers Stadium
1288,01-09-2006,Australia,South Africa,Australia,Brisbane Cricket Ground
1289,10-21-2005,South Africa,New Zealand,New Zealand,New Wanderers Stadium
1290,6-13-2005,England,Australia,England,The Rose Bowl


In [7]:
#dropping columns that wll not affect match outcomes
df_teams_2022 = df_teams.drop(['Date', 'Venue'], axis=1)
df_teams_2022.head(1571)

,Team1,Team2,Winner
0,Sri Lanka,Australia,Australia
1,India,West Indies,India
7,Sri Lanka,Australia,Australia
9,Ireland,Nepal,Ireland
10,Australia,Sri Lanka,tie
...,...,...,...
1286,South Africa,Australia,South Africa
1288,Australia,South Africa,Australia
1289,South Africa,New Zealand,New Zealand
1290,England,Australia,England


In [8]:
#Adding column 'Count': It will record the team which won the most matches from its previous 6 matches (i.e. if India have won 4 matches from its previous 6 matches and Pakistan have won 3 matches than count which select India)
# and 'Rank': The team which is having higher rank will be named in this column
df_teams_2022['Count']=0
df_teams_2022['Rank']=0
df_team_2022= df_teams_2022.reset_index(drop=True) 
df_team_2022

,Team1,Team2,Winner,Count,Rank
0,Sri Lanka,Australia,Australia,0,0
1,India,West Indies,India,0,0
2,Sri Lanka,Australia,Australia,0,0
3,Ireland,Nepal,Ireland,0,0
4,Australia,Sri Lanka,tie,0,0
...,...,...,...,...,...
1566,South Africa,Australia,South Africa,0,0
1567,Australia,South Africa,Australia,0,0
1568,South Africa,New Zealand,New Zealand,0,0
1569,England,Australia,England,0,0


In [9]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2022= df_teams_2022.reset_index(drop=True)
df_teams_2022.loc[df_teams_2022.Winner == df_teams_2022.Team1,'winning_team']=1
df_teams_2022.loc[df_teams_2022.Winner == df_teams_2022.Team2, 'winning_team']=2
df_teams_2022 = df_teams_2022.drop(['winning_team'], axis=1)

df_teams_2022.head(10)

,Team1,Team2,Winner,Count,Rank
0,Sri Lanka,Australia,Australia,0,0
1,India,West Indies,India,0,0
2,Sri Lanka,Australia,Australia,0,0
3,Ireland,Nepal,Ireland,0,0
4,Australia,Sri Lanka,tie,0,0
5,Australia,Sri Lanka,Australia,0,0
6,England,West Indies,England,0,0
7,England,West Indies,England,0,0
8,England,West Indies,West Indies,0,0
9,Ireland,USA,Ireland,0,0


In [10]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2022, prefix=['Team1', 'Team2'], columns=['Team1', 'Team2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
final.describe()

,Count,Rank,Team1_ Afghanistan,Team1_ Australia,Team1_ Bangladesh,Team1_ Bermuda,Team1_ Canada,Team1_ England,Team1_ Hong Kong,Team1_ India,...,Team2_ Pakistan,Team2_ Papua New Guinea,Team2_ Scotland,Team2_ Singapore,Team2_ South Africa,Team2_ Sri Lanka,Team2_ UAE,Team2_ USA,Team2_ West Indies,Team2_ Zimbabwe
count,1571.0,1571.0,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,...,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000
mean,0.0,0.0,0.070019,0.105029,0.078931,0.000637,0.003183,0.096754,0.003183,0.098663,...,0.144494,0.004456,0.017823,0.000637,0.078294,0.108848,0.007638,0.000637,0.060471,0.063654
std,0.0,0.0,0.255260,0.306688,0.269716,0.025230,0.056343,0.295716,0.056343,0.298304,...,0.351702,0.066624,0.132350,0.025230,0.268719,0.311547,0.087091,0.025230,0.238433,0.244213
min,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 

y_pred=rf.predict(X_test)
score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.693
Test set accuracy:  0.591


In [13]:
#adding ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
ranking = pd.read_csv('Current ICC_rankings.csv') 
fixtures = pd.read_csv('T20_WorldCup-2022 Schedule.csv')

# List for storing the group stage games
pred_set = []

In [14]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team1'].map(ranking.set_index('Team')['Rank']))
fixtures.insert(2, 'second_position', fixtures['Team2'].map(ranking.set_index('Team')['Rank']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.head(33)

,Date,first_position,second_position,Column1,Team1,Team2,Venue,Result
0,22nd October 2022,NaN,NaN,1,New Zealand,Australia,Sydney Cricket Ground,NaN
1,22nd October 2022,NaN,NaN,1,England,Afghanistan,Perth Stadium,NaN
2,23rd October 2022,NaN,NaN,1,Sri Lanka,Ireland,Bellerive Oval,NaN
3,23rd October 2022,NaN,NaN,1,India,Pakistan,Melbourne Cricket Ground,NaN
4,24th October 2022,NaN,NaN,1,Bangladesh,Netherland,Bellerive Oval,NaN
5,24th October 2022,NaN,NaN,1,South Africa,Zimbabwe,Bellerive Oval,NaN
6,25th October 2022,NaN,NaN,1,Australia,Sri Lanka,Perth Stadium,NaN
7,26th October 2022,NaN,NaN,1,England,Ireland,Melbourne Cricket Ground,NaN
8,26th October 2022,NaN,NaN,1,New Zealand,Afghanistan,Melbourne Cricket Ground,NaN
9,27th October 2022,NaN,NaN,1,Bangladesh,South Africa,Sydney Cricket Ground,NaN


In [15]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team1': row['Team1'], 'Team2': row['Team2'], 'winning_team': None})
    else:
        pred_set.append({'Team1': row['Team2'], 'Team2': row['Team1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head(33)

,Team1,Team2,winning_team
0,Australia,New Zealand,None
1,Afghanistan,England,None
2,Ireland,Sri Lanka,None
3,Pakistan,India,None
4,Netherland,Bangladesh,None
5,Zimbabwe,South Africa,None
6,Sri Lanka,Australia,None
7,Ireland,England,None
8,Afghanistan,New Zealand,None
9,South Africa,Bangladesh,None


In [16]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team1', 'Team2'], columns=['Team1', 'Team2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)

In [17]:
#group matches 
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    print("")

New Zealand  and  Australia
Winner: New Zealand 

England  and  Afghanistan
Winner: England 

Sri Lanka  and  Ireland
Winner: Sri Lanka 

India  and  Pakistan
Winner: India 

Bangladesh  and  Netherland
Winner: Bangladesh 

South Africa  and  Zimbabwe
Winner: South Africa 

Australia  and  Sri Lanka
Winner: Australia 

England  and  Ireland
Winner: England 

New Zealand  and  Afghanistan
Winner: New Zealand 

Bangladesh  and  South Africa
Winner: Bangladesh 

India  and  Netherland
Winner: India 

Pakistan  and  Zimbabwe
Winner: Pakistan 

Afghanistan  and  Ireland
Winner: Afghanistan 

England  and  Australia
Winner: England 

New Zealand  and  Sri Lanka
Winner: New Zealand 

Bangladesh  and  Zimbabwe
Winner: Bangladesh 

Pakistan  and  Netherland
Winner: Pakistan 

India  and  South Africa
Winner: India 

Australia  and  Ireland
Winner: Australia 

Afghanistan  and  Sri Lanka
Winner: Afghanistan 

England  and  New Zealand
Winner: England 

Zimbabwe  and  Netherland
Winner: Zimbabwe 

In [18]:
# List of tuples before 
semi = [('New Zealand', 'Pakistan'),
            ('India', 'England')]

In [19]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Rank'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Rank'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team1': matches[j][0], 'Team2': matches[j][1]})
        else:
            dict1.update({'Team1': matches[j][1], 'Team2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team1', 'Team_2'], columns=['Team1', 'Team2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [20]:
clean_and_predict(semi, ranking, final, rf)

New Zealand and Pakistan
Winner: Pakistan

England and India
Winner: India



In [21]:
# Finals
finals = [('India', 'Pakistan')]

In [22]:
clean_and_predict(finals, ranking, final, rf)

Pakistan and India
Winner: India

